## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [16]:
class myModel:
    def __init__(self):
        self.W1 = tf.Variable(tf.random.normal([784, 128]))
        self.b1 = tf.Variable(tf.zeros([128]))
        self.W2 = tf.Variable(tf.random.normal([128, 128]))
        self.b2 = tf.Variable(tf.zeros([128]))
        self.W3 = tf.Variable(tf.random.normal([128, 10]))
        self.b3 = tf.Variable(tf.zeros([10]))
        
    def __call__(self, x):
        x = tf.reshape(x, [-1, 784])
        
        h1 = tf.matmul(x, self.W1) + self.b1
        h1 = tf.nn.relu(h1)

        h2 = tf.matmul(h1, self.W2) + self.b2
        h2 = tf.nn.relu(h2)
        
        logits = tf.matmul(h2, self.W3) + self.b3
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [17]:
train_data, test_data = mnist_dataset()
for epoch in range(250):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32),
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1006.2664 ; accuracy 0.091233335
epoch 1 : loss 575.77106 ; accuracy 0.10978334
epoch 2 : loss 437.98746 ; accuracy 0.12683333
epoch 3 : loss 360.62366 ; accuracy 0.17588334
epoch 4 : loss 308.5096 ; accuracy 0.21695
epoch 5 : loss 270.47986 ; accuracy 0.25638333
epoch 6 : loss 241.54036 ; accuracy 0.29188332
epoch 7 : loss 218.59613 ; accuracy 0.32335
epoch 8 : loss 199.92265 ; accuracy 0.35066667
epoch 9 : loss 184.40196 ; accuracy 0.37668332
epoch 10 : loss 171.32971 ; accuracy 0.39835
epoch 11 : loss 160.17006 ; accuracy 0.41918334
epoch 12 : loss 150.5837 ; accuracy 0.43796667
epoch 13 : loss 142.26149 ; accuracy 0.45451668
epoch 14 : loss 134.94055 ; accuracy 0.47086668
epoch 15 : loss 128.48756 ; accuracy 0.48663333
epoch 16 : loss 122.77606 ; accuracy 0.50005
epoch 17 : loss 117.66975 ; accuracy 0.5126
epoch 18 : loss 113.077866 ; accuracy 0.5237167
epoch 19 : loss 108.91555 ; accuracy 0.53433335
epoch 20 : loss 105.13657 ; accuracy 0.54363334
epoch 21 : loss 101